In [ ]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import tweepy
import time
import seaborn as sns

# Twitter API Keys
consumer_key = "0YMgehLA0fffLLU8ymwXCI93J"
consumer_secret = "mBflJdnU6MIK46oul0DXTAnaAv8Bw9TvqDmF8PBtjn6Stm6D3d"
access_token = "937016914504663040-MDEJRJnYOIPdjB10kHP2faSlfQ3AzwK"
access_token_secret = "CWs0MS4EJHGOmXKHYGcAuPmqEohcDrxiexKoSDoxEtKIM"

# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())


In [ ]:
# Target User Account
target_user = ("@BBC", "@CBS", "@CNN", "@Fox", "@nytimes")


# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
sentiments = []

for target in target_user:
    
    # Variables for holding sentiments
    counter = 1
    #compound_list = []    
    #positive_list = []
    #negative_list = []
    #neutral_list = []


# Loop through 5 pages of tweets (total 100 tweets)
    for x in range(5):

        # Get all tweets from home feed
        tweets = api.user_timeline(target, page = x+1)

        # Loop through all tweets
        for tweet in tweets:
           
            # Run Vader Analysis on each tweet
            compound = analyzer.polarity_scores(tweet["text"])["compound"]
            pos = analyzer.polarity_scores(tweet["text"])["pos"]
            neu = analyzer.polarity_scores(tweet["text"])["neu"]
            neg = analyzer.polarity_scores(tweet["text"])["neg"]

            # Add sentiments for each tweet into an array
            sentiments.append({"User": target, 
                               "Date": tweet["created_at"], 
                               "Tweet": tweet["text"],
                               "Compound": compound,
                               "Positive": pos,
                               "Negative": neu,
                               "Neutral": neg,
                               "Tweets Ago": counter})

            counter = counter + 1
   


In [ ]:
# Convert sentiments to DataFrame
sentiments_pd = pd.DataFrame.from_dict(sentiments)

In [ ]:
sentiments_pd.head(500)

In [ ]:
# Create plot
sns.lmplot(x = "Tweets Ago", y = "Compound", data = sentiments_pd, fit_reg = False, hue = "User", legend = True)
sns.set_style("dark")
# Incorporate the other graph properties
plt.title("Sentiment Analysis of Tweets (%s) for %s" % (time.strftime("%x"), target_user))
plt.ylabel("Tweet Polarity")
plt.xlabel("Tweets Ago")
plt.grid()
plt.savefig("scatter_plot.png", bbox_inches="tight")
plt.show()

In [ ]:
compound_avg = pd.DataFrame(sentiments_pd.groupby("User").mean()["Compound"], index = None).reset_index()

sns.barplot(x = "User", y = "Compound", data = sentiments_pd, ci = 0)
plt.title("Average Sentiment Analysis of Tweets (%s) for %s" % (time.strftime("%x"), target_user))
plt.savefig("bar_plot.png", bbox_inches="tight")
plt.show()

In [ ]:
print(compound_avg)
print("The overall average sentiment score for the last 100 tweets, for each news source, is: %s" % round(sentiments_pd["Compound"].mean(),6))

In [ ]:
sentiments_pd.to_csv("sentiments.csv", sep=",")

In [ ]:
# 3 Observations

#1. CBS has the greatest compund score meaning that they mostly tweet out positively.
#2. CNN sends out more negative tweets than the other news agencies.
#3. Out of the five news sources CBS has the least negative comments.